In [1]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)

In [2]:
df=pd.read_excel('◎位置情報と対応(水道水) 基準満たしているか否か.xlsx')
df=df.dropna(axis=0)

In [3]:
y=df['EColi.']
X=df.loc[:, ['陰影起伏', '曲率', '傾斜角', '傾斜方向','高度', '近くの川への距離', '鉄道ポイントへの距離', '主要道路への距離', 
             'near_distance_syorizyou', 'supply_hours', 'no water days', 'total population ', 'population served', 'popu-served', 
             'number taps', 'pipes(km)', 'pipes/tabs \n個/km', 'served/pipes', '(popu-served)/pipes\n  /km', 
             'Age of oldeset\npipe ', 'ST', 'RSF', 'FL', 'PF', 'RF']]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [5]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

In [6]:
def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1

In [7]:
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

# SVC

# ロジスティック回帰

# random forest

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.77
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
#ランダムフォレストの実行
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    dic[forest]=score1

               if score1>max_score:
                    val=cross_val_score(estimator=forest,
                                        X=X_train, y=y_train,
                                        cv=3)
                    val1=np.mean(val)
                    print('params: %s, score: %3f, val: %3f' %(forest, score1, val1))



params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=17,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=21,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=25,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=29,
                       random_state=1), score: 0.772727, val: 0.598431
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=36,
                       random_state=1), score: 0.772727, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=40,
                       random_state=1), score: 0.772727, val: 0.598954
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimato

# Boosted forest